In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import time
import re

In [2]:
def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

In [3]:
numbers=re.compile(r'(\d+)')
def numericalsort(value):
    parts=numbers.split(value)
    parts[1::2]=map(int, parts[1::2])
    return parts


In [4]:
def lbp_calculated_pixel_x_y(img, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val   

In [5]:
# calculate orthogonal LBPx_y for all pixels
def lbp_calculated_pixel_ortho_x_y(img_stream,frame_count, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    index=frame_count
    # for X-Y place
    center = img_stream[index][x][y]
    val_ar = []
    val_ar.append(get_pixel(img_stream[index], center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img_stream[index], center, x, y+1))       # right
    val_ar.append(get_pixel(img_stream[index], center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img_stream[index], center, x+1, y))       # bottom
    val_ar.append(get_pixel(img_stream[index], center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img_stream[index], center, x, y-1))       # left
    val_ar.append(get_pixel(img_stream[index], center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img_stream[index], center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val   


In [6]:
# calculate orthogonal LBP_x_t_for all pixels
# for X-T plane
def lbp_calculated_pixel_ortho_x_t(img_stream,frame_count, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    index=frame_count
    
    center = img_stream[index][x][y]
    val_ar = []
    # top right is x+1, y, T+1
    new_value = 0
    try:
        if img_stream[index+1][x+1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
    
    # right is x+1, y, T
    new_value = 0
    try:
        if img_stream[index][x+1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # bottom right is x+1, y, T-1
    new_value = 0
    try:
        if img_stream[index-1][x+1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # bottom is x,y, T-1
    new_value = 0
    try:
        if img_stream[index-1][x][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
        
    # bottom left is x-1, y, T-1
    new_value = 0
    try:
        if img_stream[index-1][x-1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
    
    # left is x-1,y, T
    new_value = 0
    try:
        if img_stream[index][x-1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # top left is x-1,y, T+1
    new_value = 0
    try:
        if img_stream[index+1][x-1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # top is x,y, T+1
    new_value = 0
    try:
        if img_stream[index+1][x][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
     
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val   


In [7]:
# calculate orthogonal LBP_y_t_for all pixels
# for Y-T plane
def lbp_calculated_pixel_ortho_y_t(img_stream,frame_count, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    index=frame_count
    
    center = img_stream[index][x][y]
    val_ar = []
    # top right is x, y+1, T+1
    new_value = 0
    try:
        if img_stream[index+1][x][y+1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
    
    # right is x, y, T+1
    new_value = 0
    try:
        if img_stream[index+1][x][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # bottom right is x, y-1, T+1
    new_value = 0
    try:
        if img_stream[index+1][x][y-1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # bottom is x,y-1, T
    new_value = 0
    try:
        if img_stream[index][x][y-1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
        
    # bottom left is x, y-1, T-1
    new_value = 0
    try:
        if img_stream[index-1][x][y-1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
    
    # left is x,y, T-1
    new_value = 0
    try:
        if img_stream[index-1][x][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # top left is x,y+1, T-1
    new_value = 0
    try:
        if img_stream[index-1][x][y+1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # top is x,y+1, T
    new_value = 0
    try:
        if img_stream[index][x][y+1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
     
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val   


In [8]:
os.chdir("F:\CA2b")
orig_dir = os.getcwd()
orig_dir

'F:\\CA2b'

In [9]:
orig_dir

'F:\\ca2b'

In [10]:
# Create the folders for each main subject
frames_source_path=os.path.join(orig_dir, 'CASME2_Extracted_Frames')
subject_list = os.listdir(frames_source_path)

for subject in subject_list:
    subject_path_list=os.path.join(frames_source_path, subject)
    file_list = os.listdir(subject_path_list)
    os.chdir("F:\CA2b\LBP_Top_full_raw")
    if not os.path.exists(subject):
        os.mkdir(subject)
        
#     subject_dir=os.getcwd()
#     os.chdir(orig_dir)
#     for folder in file_list:
#         print ("current folder is ", folder)
#         folder_file=os.path.join(subject_path_list, folder)
#         print ("current folder path is", folder_file)
#         # extract vectors
#         print ("creating a new folder for ", folder, "in subject", subject)
#         os.chdir("F:\CA2\LBP_TOP_full")
#         if not os.path.exists(subject):
#             os.mkdir(subject)
#             print ("created subdirectory for subject", subject)
        
        
#         loadedvideo = imageio.get_reader(video_file, 'ffmpeg')
#          print ("video loaded")
#         framecount=loadedvideo.count_frames()
#         print ("subject-", subject, ",video-", video, ",framecount-", framecount)
#          print ("number of frames in video=", framecount)
#         frames=[]
#         framerange = [x for x in range(framecount)]
#         for frame in framerange:
#             image = loadedvideo.get_data(frame)
#             frames.append(image)
#         print ("frames extracted")
#         fname=video.split('.')[0]
#          print ("new folder name to be created", fname)
#         os.chdir("F:\CA2bojireddy\CASME2_Extracted_Frames")
#          print ("changed directory to extracted frames")
#         ext_dir=os.getcwd()
#          print ("current directory is", ext_dir)
#         if not os.path.exists(subject):
#             os.mkdir(subject)
#              print ("created subdirectory for subject", subject)
#         write_subject_path=os.path.join(ext_dir, subject)
#          print ("path for writing in subject path",write_subject_path)
#         os.chdir(write_subject_path)
#          print ("changed directory to subject level")
#         subject_dir=os.getcwd()
#          print ("current directory is", subject_dir)
#         if not os.path.exists(fname):
#             os.mkdir(fname)
#              print ("created subdirectory for video", video)
#         write_video_path=os.path.join(write_subject_path,fname)
#         print ("video path created for video", fname)
#         print ("and the path is ", write_video_path)
#         os.chdir(write_video_path)
#         video_dir=os.getcwd()
#         print ("changed directory to video level")
#         print ("current directory is", video_dir)
#         print ("will be writing the frames here")
#         count=1
#         for image in frames:
#             filename="frame"+str(count)+".jpg"
#             cv2.imwrite(filename,image)
#             count=count+1

os.chdir(orig_dir)











# video_path=r"F:\CA2b\Happy_Frames\EP02_01f"
# frame_list=os.listdir(video_path)
# count=0
# image_sequence=[]
# for image in frame_list:
#     image_file=os.path.join(video_path, image)
#     img_bgr = cv2.imread(image_file)
#     height, width, channel = img_bgr.shape
#     img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
#     image_sequence.append(img_gray)

    
    
    


In [10]:

###### EXTRACT LBP FEATURE


start_time=time.time()
# Extract from each subfolder and create directory to store info
#sub_list=subject_list[0:1]
frames_source_path=os.path.join(orig_dir, 'CASME2_RAW_selected')
subject_list = os.listdir(frames_source_path)
sub_list=subject_list[20:]
for subject in sub_list:
    subject_path_list=os.path.join(frames_source_path, subject)
    file_list = os.listdir(subject_path_list)
    #sub_file_list=file_list[0:1]
    for folder in file_list:
        print ("current folder is ", folder)
        folder_file=os.path.join(subject_path_list, folder)
        print ("current folder path is", folder_file)
        frame_list=os.listdir(folder_file)
                     
        # extract vectors
        # Get the frame volume sequence
        count=0
        image_sequence=[]
        #for image in frame_list:
        for image in sorted(frame_list, key=numericalsort):
            #print ("processing image", image, "in folder, ", folder, " for subject", subject)
            image_file=os.path.join(folder_file, image)
            img_bgr = cv2.imread(image_file)
            height, width, channel = img_bgr.shape
            img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
            image_sequence.append(img_gray)
        
        # extract all the features from the sequence    
        start_time=time.time()
        height, width = image_sequence[0].shape
        count=0
        print("sequence processing started for", folder)
        hist_lbp_top=[]
        for image in image_sequence:
            img_lbp_x_y = np.zeros((height, width), np.uint8)
            img_lbp_x_t = np.zeros((height, width), np.uint8)
            img_lbp_y_t = np.zeros((height, width), np.uint8)
            for i in range(0, height):
                for j in range(0, width):
                    img_lbp_x_y[i,j]=lbp_calculated_pixel_ortho_x_y(image_sequence,count, i, j)
                    img_lbp_x_t[i,j]=lbp_calculated_pixel_ortho_x_t(image_sequence,count, i, j)
                    img_lbp_y_t[i,j]=lbp_calculated_pixel_ortho_y_t(image_sequence,count, i, j)
            hist_lbp_x_y = cv2.calcHist([img_lbp_x_y], [0], None, [256], [0, 256])
            hist_lbp_x_t = cv2.calcHist([img_lbp_x_t], [0], None, [256], [0, 256])
            hist_lbp_y_t = cv2.calcHist([img_lbp_y_t], [0], None, [256], [0, 256])
            histogram=np.concatenate((hist_lbp_x_y, hist_lbp_x_t, hist_lbp_y_t), axis=0)
            hist_lbp_top.append(histogram)
            count=count+1

        print("sequence processing finished for", folder)
        print("--- %s seconds ---" % (time.time() - start_time))
        
        print ("creating a new folder for ", folder, "in subject", subject)
        os.chdir("F:\CA2b\LBP_Top_full_raw")
        location=os.getcwd()
        new_subject_path=os.path.join(location, subject)
        print ("at copy location folder", new_subject_path)
        os.chdir(new_subject_path)
        if not os.path.exists(folder):
            os.mkdir(folder)
            print ("created subdirectory for folder", folder)
        new_folder_path=os.path.join(new_subject_path, folder)
        os.chdir(new_folder_path)
        print (" at file saving location", new_folder_path)
        #hist_lbp_top=[1]
        filename="hist_lbptop_raw.npy"
        save_file=os.path.join(new_folder_path, filename)
        location=save_file
        print("save file name", save_file)
        np.save(save_file,hist_lbp_top)
        print("file saved")

            
            
os.chdir(orig_dir)   

current folder is  EP01_07
current folder path is F:\CA2b\CASME2_RAW_selected\sub21\EP01_07
sequence processing started for EP01_07
sequence processing finished for EP01_07
--- 159.76180934906006 seconds ---
creating a new folder for  EP01_07 in subject sub21
at copy location folder F:\CA2b\LBP_Top_full_raw\sub21
created subdirectory for folder EP01_07
 at file saving location F:\CA2b\LBP_Top_full_raw\sub21\EP01_07
save file name F:\CA2b\LBP_Top_full_raw\sub21\EP01_07\hist_lbptop_raw.npy
file saved
current folder is  EP05_02
current folder path is F:\CA2b\CASME2_RAW_selected\sub21\EP05_02
sequence processing started for EP05_02
sequence processing finished for EP05_02
--- 452.19669127464294 seconds ---
creating a new folder for  EP05_02 in subject sub21
at copy location folder F:\CA2b\LBP_Top_full_raw\sub21
created subdirectory for folder EP05_02
 at file saving location F:\CA2b\LBP_Top_full_raw\sub21\EP05_02
save file name F:\CA2b\LBP_Top_full_raw\sub21\EP05_02\hist_lbptop_raw.npy
fil

sequence processing started for EP01_08
sequence processing finished for EP01_08
--- 478.563086271286 seconds ---
creating a new folder for  EP01_08 in subject sub24
at copy location folder F:\CA2b\LBP_Top_full_raw\sub24
created subdirectory for folder EP01_08
 at file saving location F:\CA2b\LBP_Top_full_raw\sub24\EP01_08
save file name F:\CA2b\LBP_Top_full_raw\sub24\EP01_08\hist_lbptop_raw.npy
file saved
current folder is  EP02_02f
current folder path is F:\CA2b\CASME2_RAW_selected\sub24\EP02_02f
sequence processing started for EP02_02f
sequence processing finished for EP02_02f
--- 174.87246537208557 seconds ---
creating a new folder for  EP02_02f in subject sub24
at copy location folder F:\CA2b\LBP_Top_full_raw\sub24
created subdirectory for folder EP02_02f
 at file saving location F:\CA2b\LBP_Top_full_raw\sub24\EP02_02f
save file name F:\CA2b\LBP_Top_full_raw\sub24\EP02_02f\hist_lbptop_raw.npy
file saved
current folder is  EP07_01
current folder path is F:\CA2b\CASME2_RAW_selected\

sequence processing started for EP03_10
sequence processing finished for EP03_10
--- 435.1885242462158 seconds ---
creating a new folder for  EP03_10 in subject sub26
at copy location folder F:\CA2b\LBP_Top_full_raw\sub26
created subdirectory for folder EP03_10
 at file saving location F:\CA2b\LBP_Top_full_raw\sub26\EP03_10
save file name F:\CA2b\LBP_Top_full_raw\sub26\EP03_10\hist_lbptop_raw.npy
file saved
current folder is  EP07_28
current folder path is F:\CA2b\CASME2_RAW_selected\sub26\EP07_28
sequence processing started for EP07_28
sequence processing finished for EP07_28
--- 223.75347018241882 seconds ---
creating a new folder for  EP07_28 in subject sub26
at copy location folder F:\CA2b\LBP_Top_full_raw\sub26
created subdirectory for folder EP07_28
 at file saving location F:\CA2b\LBP_Top_full_raw\sub26\EP07_28
save file name F:\CA2b\LBP_Top_full_raw\sub26\EP07_28\hist_lbptop_raw.npy
file saved
current folder is  EP07_37
current folder path is F:\CA2b\CASME2_RAW_selected\sub26\E

In [12]:
location="F:\CA2b\LBP_Top_full_raw\sub09\EP09f\hist_lbptop_raw.npy"
hist=np.load(location) 


In [13]:
hist[0].shape

(768, 1)

In [ ]:
# check saves files
hist=[]
saved_location=os.path.join(orig_dir, 'LBP_TOP_full')
folders=['sub01']
for folder in folders:
    filepath=os.path.join(saved_location, folder)
    sub_folders=os.listdir(filepath)
    for sub_folder in sub_folders:
        subfolder_path=os.path.join(filepath, sub_folder)
        final_path=os.path.join(subfolder_path, "hist_lbptop.npy")
        print (final_path)
        histogram = np.load(final_path)
        hist.append(histogram)
        
# hist2=np.load(r"F:\CA2b\LBP_TOP_fu 

In [ ]:
hist[0].shape

In [ ]:
# Create LBPTOP Histogram for each image in the volume sequence
start_time=time.time()  
height, width = image_sequence[0].shape
count=0
hist_lbp_top=[]
for image in image_sequence:
    img_lbp_x_y = np.zeros((height, width), np.uint8)
    img_lbp_x_t = np.zeros((height, width), np.uint8)
    img_lbp_y_t = np.zeros((height, width), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
            img_lbp_x_y[i,j]=lbp_calculated_pixel_ortho_x_y(image_sequence,count, i, j)
            img_lbp_x_t[i,j]=lbp_calculated_pixel_ortho_x_t(image_sequence,count, i, j)
            img_lbp_y_t[i,j]=lbp_calculated_pixel_ortho_y_t(image_sequence,count, i, j)
    hist_lbp_x_y = cv2.calcHist([img_lbp_x_y], [0], None, [256], [0, 256])
    hist_lbp_x_t = cv2.calcHist([img_lbp_x_t], [0], None, [256], [0, 256])
    hist_lbp_y_t = cv2.calcHist([img_lbp_y_t], [0], None, [256], [0, 256])
    histogram=np.concatenate((hist_lbp_x_y, hist_lbp_x_t, hist_lbp_y_t), axis=0)
    hist_lbp_top.append(histogram)
    count=count+1
    
    
print("--- %s seconds ---" % (time.time() - start_time))
    

In [ ]:
hist_lbp_top[0].shape